In [1]:
import pyodbc

# إعدادات الاتصال بـ SQL Server باستخدام Windows Authentication
server = r'DESKTOP-5UTDA3H\GUJJKKKK'  # اسم الخادم
database = 'PipedriveDB'  # اسم قاعدة البيانات

def create_database_and_table(server, database):
    # إنشاء اتصال بـ SQL Server مع تمكين autocommit
    conn = pyodbc.connect(
        f'DRIVER={{SQL Server}};SERVER={server};Trusted_Connection=yes;',
        autocommit=True
    )
    cursor = conn.cursor()

    # إنشاء قاعدة البيانات إذا لم تكن موجودة
    try:
        cursor.execute(f"IF DB_ID(N'{database}') IS NULL CREATE DATABASE {database};")
        print(f"Database '{database}' is ready.")
    except Exception as e:
        print(f"Error creating database: {e}")
    finally:
        cursor.close()
        conn.close()

    # الاتصال بقاعدة البيانات الجديدة
    conn = pyodbc.connect(
        f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
    )
    cursor = conn.cursor()

    # إنشاء جدول لتخزين الأنشطة
    create_table_query = '''
    IF OBJECT_ID('dbo.Activities', 'U') IS NULL
    CREATE TABLE dbo.Activities (
        id INT IDENTITY(1,1) PRIMARY KEY,
        user_name NVARCHAR(100),
        activity_subject NVARCHAR(255),
        activity_type NVARCHAR(100),
        due_date DATE
    );
    '''
    try:
        cursor.execute(create_table_query)
        conn.commit()
        print("Table 'Activities' is ready.")
    except Exception as e:
        print(f"Error creating table: {e}")
    finally:
        cursor.close()
        conn.close()

# تنفيذ الدالة لإنشاء قاعدة البيانات والجدول
create_database_and_table(server, database)


Database 'PipedriveDB' is ready.
Table 'Activities' is ready.


In [2]:
import requests
from datetime import datetime, timedelta

# إعدادات Pipedrive لكل مستخدم
users = {
    'Kate Rawlings': {
        'user_id': 9849043,
        'api_token': 'abd20c1b91fd70632e68535de75e52032de98117'
    },
    'Daniel Marrone': {
        'user_id': 14469851,
        'api_token': 'b14a1daf8afd2fad9c2cded0e53632cb77dac0da'
    },
    'Patrick Goldsmith': {
        'user_id': 12690568,
        'api_token': '3f7153409c3e735d6017d2fd4bef0eb2b528c798'
    },
    'Quincy Smith': {
        'user_id': 14726019,
        'api_token': 'abd20c1b91fd70632e68535de75e52032de98117'
    }
}

base_url = 'https://entro-56dbfc.pipedrive.com/api/v1'

def get_activities_for_current_week():
    all_activities = {}
    today = datetime.today()
    start_of_week = today - timedelta(days=today.weekday())  # يوم الاثنين
    end_of_week = start_of_week + timedelta(days=6)  # يوم الأحد

    start_date_str = start_of_week.strftime('%Y-%m-%d')
    end_date_str = end_of_week.strftime('%Y-%m-%d')

    for user, user_info in users.items():
        user_id = user_info['user_id']
        api_token = user_info['api_token']
        
        url = f'{base_url}/activities'
        params = {
            'user_id': user_id,
            'start_date': start_date_str,
            'end_date': end_date_str,
            'api_token': api_token
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            activities = data.get('data', [])
            all_activities[user] = activities
            print(f"Retrieved {len(activities)} activities for {user}.")
        else:
            print(f"Failed to retrieve activities for {user}. Status code: {response.status_code}")
            all_activities[user] = []
    return all_activities

def insert_activities_into_db(server, database, activities):
    conn = pyodbc.connect(
        f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
    )
    cursor = conn.cursor()

    insert_query = '''
    INSERT INTO dbo.Activities (user_name, activity_subject, activity_type, due_date)
    VALUES (?, ?, ?, ?)
    '''

    try:
        for user, user_activities in activities.items():
            for activity in user_activities:
                cursor.execute(
                    insert_query,
                    user,
                    activity.get('subject', 'No Subject'),
                    activity.get('type', 'No Type'),
                    activity.get('due_date')
                )
        conn.commit()
        print("All activities have been inserted successfully.")
    except Exception as e:
        print(f"Error inserting activities: {e}")
    finally:
        cursor.close()
        conn.close()

if __name__ == "__main__":
    activities = get_activities_for_current_week()
    insert_activities_into_db(server, database, activities)


Retrieved 17 activities for Kate Rawlings.
Retrieved 44 activities for Daniel Marrone.
Retrieved 48 activities for Patrick Goldsmith.
Retrieved 100 activities for Quincy Smith.
All activities have been inserted successfully.


In [3]:
conn = pyodbc.connect(
    f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
)
cursor = conn.cursor()

# استعلام عن جميع الأنشطة في الجدول
cursor.execute("SELECT * FROM dbo.Activities")
rows = cursor.fetchall()

for row in rows:
    print(row)

cursor.close()
conn.close()


(1, 'Kate Rawlings', 'FT Ramin - Halston', 'important_call', '2024-08-26')
(2, 'Kate Rawlings', 'Phone call from Alex', 'important_call', '2024-08-26')
(3, 'Kate Rawlings', 'Anthony & Kosta', 'architect_presentation1', '2024-08-27')
(4, 'Kate Rawlings', 'Ramin', 'important_call', '2024-08-27')
(5, 'Kate Rawlings', 'Anthony & Kosta', 'architect_presentation1', '2024-08-28')
(6, 'Kate Rawlings', 'Meeting with Anh', 'architect_presentation1', '2024-08-28')
(7, 'Kate Rawlings', 'Beatrice', 'important_call', '2024-08-28')
(8, 'Kate Rawlings', 'Michael', 'important_call', '2024-08-28')
(9, 'Kate Rawlings', 'Phone call from Alex', 'important_call', '2024-08-28')
(10, 'Kate Rawlings', 'Stan', 'important_call', '2024-08-28')
(11, 'Kate Rawlings', 'Amir', 'important_call', '2024-08-28')
(12, 'Kate Rawlings', 'Phone call from Ivan', 'important_call', '2024-08-28')
(13, 'Kate Rawlings', 'Phone call from Stephen', 'important_call', '2024-08-28')
(14, 'Kate Rawlings', 'Meeting with Vernon', 'archite